# TP : LLM, Langchain et RAG
## Installation des dépendances

### Prérequis : 
- Mettre la clé d'API OpenAI en variable d'environnement
- Installer les packages suivants : 

In [ ]:
!pip install langchain openai datasets transformers PyMuPDF langchain_community langchain_openai langchain_text_splitters faiss-cpu sentence-transformers

## LLM
### OpenAI
**Objectif** : faire une requête au modèle 'gpt4o-mini' via la librairie d'OpenAI et récupérer : 
- la réponse 
- le nombre de tokens d'entrée
- le nombre de tokens de sorties

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
def ask_gpt(question: str):
    """
    Prend une question et retourne la réponse ainsi que les KPIs d'usage
    
    Args: 
        question(str) : question à poser
    """
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"{question}"}
      ]
    )
    
    return(completion.choices[0].message.content, completion.usage)

answer, usage = ask_gpt("Qu'est ce qu'un LLM ?")

In [3]:
usage

CompletionUsage(completion_tokens=239, prompt_tokens=25, total_tokens=264, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [4]:
answer

'Un LLM, ou "Large Language Model" (modèle de langage de grande taille en français), est un type de modèle d\'intelligence artificielle conçu pour comprendre et générer du texte en langage naturel. Ces modèles sont entraînés sur d\'énormes ensembles de données textuelles provenant de diverses sources, ce qui leur permet d\'apprendre les structures des langues, le vocabulaire, et même le contexte dans lequel les mots se rejoignent.\n\nLes LLM fonctionnent généralement grâce à des architectures de type réseau de neurones, tels que les transformateurs, qui leur permettent de traiter des séquences de texte et de créer des prédictions sur le mot suivant dans une phrase, entre autres tâches.\n\nLes applications des LLM sont variées, allant de la génération de texte (comme écrire des articles ou des histoires) à la traduction automatique, en passant par la conversation et l\'assistance dans diverses tâches linguistiques.\n\nDes exemples connus de LLM incluent GPT-3 et GPT-4 d\'OpenAI, BERT de

## RAGS from Scracth
**Objectif** : Développer, from scratch, une architecture RAG en 4 modules: 
- Lecture et découpage d'un document PDF
- Générer les embeddings
- Récupérer les passages pertinents en fonction d'une requête
- Générer une réponse à partir de la question et des passages pertinents

In [6]:
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import PyPDF2
import os

/Users/axel/Documents/Nexialog/Formation LLM/TP/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
# Module 1 : Lecture et découpage d’un document PDF
def read_and_split_pdf(pdf_path, chunk_size=500):
    """
    Lit un fichier PDF et découpe le contenu en morceaux de texte.
    
    Args:
    pdf_path (str): Chemin du fichier PDF.
    chunk_size (int): Taille des morceaux (en nombre de caractères).
    
    Returns:
    list: Liste des morceaux de texte.
    """
    text_chunks = []
    with open(pdf_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text()
        text_chunks = [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size)]
    return text_chunks

# Tester avec un fichier PDF
pdf_chunks = read_and_split_pdf("../pdf/Cours_LLM.pdf")
print(f"Nombre de morceaux générés : {len(pdf_chunks)}")
print("Extrait du premier morceau :\n", pdf_chunks[0])

Nombre de morceaux générés : 87
Extrait du premier morceau :
 Large Language Models
Introduction
Les modèles de langage de grande taille (LLM) représentent une avancée majeure dans le domaine
de l’intelligence artificielle, avec des applications variées allant de la génération de texte à l’analyse
sémantique et à la traduction automatique. En s’appuyant sur des architectures de réseaux neuronaux,
notamment les modèles Transformers, les LLM permettent de traiter, comprendre et générer du langage
naturel de manière performante et cohérente.
Nous explorerons 


In [8]:
# Module 2 : Génération des embeddings et stockage avec FAISS

def generate_embeddings(chunks):
    """
    Génère les embeddings pour chaque morceau de texte.
    
    Args:
        chunks (list): Liste des morceaux de texte.
    
    Returns:
        np.array: Tableau numpy des embeddings.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(chunks, show_progress_bar=True)
    return np.array(embeddings)

def create_faiss_index(embeddings):
    """
    Crée un index FAISS à partir des embeddings.
    
    Args:
    embeddings (np.array): Tableau numpy des embeddings.
    
    Returns:
    faiss.IndexFlatL2: Index FAISS.
    """
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

# Générer les embeddings et créer l'index
embeddings = generate_embeddings(pdf_chunks)
faiss_index = create_faiss_index(embeddings)

print(f"Nombre de vecteurs dans l'index : {faiss_index.ntotal}")

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Nombre de vecteurs dans l'index : 87


In [9]:
# Module 3 : Recherche des passages pertinents avec une requête
def query_faiss_index(query, index, model, chunks, k=5):
    """
    Recherche les passages les plus pertinents dans l'index FAISS.
    
    Args:
    query (str): La requête de l'utilisateur.
    index (faiss.IndexFlatL2): L'index FAISS.
    model (SentenceTransformer): Modèle pour générer les embeddings.
    chunks (list): Liste des morceaux de texte.
    k (int): Nombre de passages à retourner.
    
    Returns:
    list: Liste des passages pertinents.
    """
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

# Tester avec une requête
query = "Qu'est ce qu'un RAG ?"
relevant_chunks = query_faiss_index(query, faiss_index, SentenceTransformer('all-MiniLM-L6-v2'), pdf_chunks)
print("Passages pertinents :")
for chunk in relevant_chunks:
    print("-" * 80)
    print(chunk)

Passages pertinents :
--------------------------------------------------------------------------------
Génération de la base de données : Pour créer la base de données sur laquelle le RAG va
s’appuyer, on transforme l’ensemble des documents par le biais d’embeddings. On représente
l’ensemble des paragraphes sous forme de vecteurs, ce qui permet de capturer des relations sé-
mantiques entre les mots et de les rendre compréhensibles par la machine.
•Récupération d’informations : Lorsqu’une requête est reçue, le RAG commence par identifier
les segments de texte pertinents dans sa base de données. Le
--------------------------------------------------------------------------------
cifiques récupérées. Cela permet de produire des réponses qui sont à la fois informatives
et particulièrement adaptées à la requête.
5.2 Avantages
L’utilisation de RAG offre plusieurs avantages significatifs :
•Amélioration de l’exactitude : En s’appuyant sur des informations précises récupérées, les
RAG peuvent f

In [10]:
# Module 4 : Générer une réponse avec un LLM
def generate_answer(query, relevant_chunks):
    """
    Génère une réponse à partir de la question et des passages pertinents.
    
    Args:
    query (str): La question posée.
    relevant_chunks (list): Passages pertinents.
    
    Returns:
    str: Réponse générée.
    """
    context = "\n".join(relevant_chunks)
    prompt = f"Voici des informations utiles :\n{context}\n\nQuestion : {query}\nRéponse :"
    
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "system", "content": "Tu es un assistant RAG spécialisé dans la rédaction de réponse."},
        {"role": "user", "content": f"{prompt}"}
      ]
    )
    
    return(completion.choices[0].message.content)

# Tester la génération de réponse
response = generate_answer(query, relevant_chunks)
print("Réponse générée :")
print(response)


Réponse générée :
Un RAG, ou "Retrieval-Augmented Generation", est un modèle de traitement du langage qui combine des capacités de récupération d'informations avec des mécanismes de génération de texte. Il fonctionne en deux étapes principales :

1. **Génération de la base de données** : Les documents pertinents sont transformés en vecteurs par le biais d'embeddings, ce qui permet de capturer les relations sémantiques entre les mots. Cela facilite la compréhension du texte par la machine.

2. **Récupération d'informations** : Lorsqu'une requête est formulée, le RAG identifie les segments de texte pertinents dans sa base de données. Il utilise ces informations contextuelles spécifiques pour générer des réponses informatives et adaptées au contexte de la question posée.

Les avantages des RAG incluent une amélioration de l'exactitude des réponses grâce à l'utilisation d'informations précises, une réduction des problèmes d'hallucination par rapport aux modèles de langage standard, et une 

## Langchain
**Objectif** : faire une requête au modèle 'gpt4o-mini' via la librairie Langchain et récupérer la réponse 

In [12]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [13]:
def ask_gpt_langchain(question):
    """
    Prend une question et retourne la réponse générée via langchain
    
    Args: 
        question(str) : question à poser
    """
        
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    prompt_template = PromptTemplate.from_template(question)
    chain = LLMChain(llm=llm, prompt=prompt_template)
    
    return(chain.invoke({}))

anwser = ask_gpt_langchain("Qu'est ce qu'un transformer ?")
print(anwser)

/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_4641/2562852100.py:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


{'text': 'Un transformer est un modèle d\'apprentissage automatique utilisé principalement dans le domaine du traitement du langage naturel (NLP). Introduit dans l\'article "Attention is All You Need" par Vaswani et al. en 2017, le transformer a révolutionné la manière dont les modèles sont construits pour traiter des séquences de données.\n\nVoici quelques caractéristiques clés des transformers :\n\n1. **Architecture basée sur l\'attention** : Contrairement aux modèles précédents, comme les réseaux de neurones récurrents (RNN), qui traitent les données de manière séquentielle, les transformers utilisent un mécanisme d\'attention qui permet de traiter toutes les entrées simultanément. Cela permet de capturer les relations entre les mots, peu importe leur position dans la séquence.\n\n2. **Encoders et Decoders** : L\'architecture d\'un transformer est généralement divisée en deux parties : l\'encodeur, qui transforme la séquence d\'entrée en une représentation interne, et le décodeur, q

## Lecture d'un PDF et utilisation dans RAG avec LangChain
**Objectif** : Construire une architecture RAG avec langchain via 3 modules : 
1 - récupération du document via PyPDFLoader
2 - génération du retriever après avoir split le document
3 - Création de la chaîne de réponse à la question


In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_openai import OpenAIEmbeddings

In [15]:
def get_docs(pdf_path):
    """
    Charge et extrait le contenu textuel d'un fichier PDF.

    Args:
        pdf_path (str): Chemin vers le fichier PDF à charger.

    Returns:
        list: Liste contenant le contenu textuel de chaque page du PDF.
    """
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    return(docs)

In [19]:
def get_retriever(docs):
    """
    Crée un récupérateur (retriever) basé sur les embeddings pour une recherche de similarité.

    Args:
        docs (list): Liste des documents à traiter

    Returns:
        BaseRetriever
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                                   chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = InMemoryVectorStore.from_documents(documents=splits, 
                                                     embedding=OpenAIEmbeddings())
    return vectorstore.as_retriever()

In [20]:
def generate_answer(question, retriever) :
    """
    Génère une réponse à une question en utilisant une architecture RAG.

    Args:
        question (str): La question posée.
        retriever (BaseRetriever): Un récupérateur permettant de chercher le contexte pertinent
                                   pour répondre à la question.

    Returns:
        str: La réponse générée par le modèle, basée sur le contexte récupéré. 
    """
    system_prompt = ("You are an assistant for question-answering tasks. "
                     "Use the following pieces of retrieved context to answer "
                     "the question. If you don't know the answer, say that you "
                     "don't know. Use three sentences maximum and keep the "
                     "answer concise."
                     "\n\n"
                     "{context}")
    
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt),
                                               ("human", "{input}"),])
    
    llm = ChatOpenAI(model="gpt-4o-mini")
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    
    results = rag_chain.invoke({"input": f"{question}"})
    
    return(results)

In [21]:
docs = get_docs('../pdf/Cours_LLM.pdf')
retriever = get_retriever(docs)
results = generate_answer("Qu'est ce qu'un RAG ?", retriever)

In [22]:
print(results["answer"])

Un RAG (Retrieval Augmented Generation) est un système qui combine la capacité de génération d'un modèle de langage (LLM) avec un mécanisme de récupération d'informations. Cela permet d'améliorer la qualité et la précision des réponses en utilisant des données spécifiques d'une vaste base de données. Les RAG sont également plus flexibles et réduisent les coûts d'entraînement en permettant des mises à jour sans réentraînement complet du modèle.


In [23]:
print(results["context"])

[Document(id='66d6b942-8604-4319-a1e0-84b775bb9ead', metadata={'source': '../pdf/Cours_LLM.pdf', 'page': 18}, page_content='5 RAG : Retrieval Augmented Generation\nLes systèmes de génération augmentée par récupération d’informations (RAG) sont une évolution\ndes LLM. Un RAG combine la capacité de génération d’un LLM avec un mécanisme de récupération\nd’informations pour améliorer la qualité et la précision des réponses du modèle. Ce mécanisme permet\nau modèle de rechercher et d’utiliser des informations spécifiques stockées dans une vaste base de données'), Document(id='a3f30ddd-1225-4cbe-97ff-60fdb8b4dfc2', metadata={'source': '../pdf/Cours_LLM.pdf', 'page': 19}, page_content='que les outils de recherche spécialisée.\n•Réduction des coûts d’entraînement : Puisque les RAG peuvent être mis à jour simplement\nen ajoutant ou en modifiant les documents dans leur base de données de récupération, ils réduisent\nles besoins en réentraînement complet du modèle de base. Cela conduit à une rédu

# Ollama

In [24]:
import ollama
response = ollama.chat(model='mistral', messages=[
   {
     'role': 'user',
     'content': 'Talk about Data Science',
   },
 ], stream=True)

"for chunk in response:    print(chunk["message"]["content"], end='', flush=True)

 Data Science is a multidisciplinary field that uses scientific methods, processes, algorithms, and systems to extract knowledge and insights from structured and unstructured data. It's an amalgamation of Statistics, Computer Science, Mathematics, and Domain Expertise. The goal is to make sense of the data, understand patterns, make predictions or decisions, and support decision-making processes through data analysis and interpretation.
Data Science involves several key stages:
1. **Data Collection**: This is the first step where raw data is gathered from various sources such as databases, files, APIs, and real-time data streams.
2. **Data Cleaning**: The collected data needs to be cleaned and preprocessed to handle missing values, outliers, inconsistencies, and other issues that can affect the quality of analysis.
3. **Data Exploration**: This step involves understanding the data by exploring its properties, generating descriptive statistics, visualizing it, and discovering initial pa